In [1]:
import numpy as np
import pandas as pd
import datetime as dt

from plastockmethods import name_the_zones, name_the_particles, frequentation_name, situation_name
from plastockmethods import particle_groups, name_the_substrate, name_the_distance, table_css_styles



def add_labels_display(data, column_name, labels):
    # Changes column values from integer to labels and sets df.index to column_name
    data[column_name] = data[column_name].apply(lambda x: labels[x])
    data.set_index(column_name, drop=True, inplace=True)
    data.index.name = None
        
    return data.style.format(precision=2).set_table_styles(table_css_styles)

# Variables aléatoires

In [2]:
c = pd.read_csv("data/end_pipe/micro_results.csv")
# column groups by material type
fibres = ['fbr', 'fbb', 'fbj', 'fbt', 'fbn', 'fba']
fdure = ['frr', 'frb', 'frj', 'frt', 'frn', 'fra']
souple = ['fsr', 'fsb', 'fsj', 'fst', 'fsn', 'fsa']
fragments = [*fdure, *souple]


# the sum of all particles for one sample
c["added"] = c[[*fragments, *fibres]].sum(axis=1)
# the sum of the different groups
c["fibres"] = c[fibres].sum(axis=1)
c["fdure"] = c[fdure].sum(axis=1)
c["souple"] = c[souple].sum(axis=1)
c["fragments"] = c[fragments].sum(axis=1)
c.rename(columns={"Echantillon":"echantillon"}, inplace=True)

# melt the data to long form
cAdded = c[['Plage', 'echantillon', 'orientation', 'position', 'substrat','frequentation', 'situation', 'distance', "fibres", "fdure", "souple", "fragments"]].copy()
cAdded[["frequentation", "situation", "distance"]] = cAdded[["frequentation", "situation", "distance"]].astype("int")

# work columns
id_vars = ['Plage', 'echantillon', 'orientation', 'position', 'substrat', 'frequentation', 'situation', 'distance']
value_vars = ["fibres", "fdure", "souple"]

# make workdata to long form
work_data = pd.melt(cAdded, id_vars=id_vars, value_vars=value_vars)
work_data.rename(columns={"variable":"objet", "value":"particules"}, inplace=True)
work_data["particules"] = work_data["particules"].astype("int")

# les plages
locations = c.Plage.unique()
nlocations = len(locations)
samples = c.echantillon.unique()
nsamples = len(samples)
tquantity = work_data.particules.sum()

def add_weight_of_samples(data, samps: int=nsamples, tquantity: int=tquantity):
    # calculates the weight of the samples and the counts for a set of data    
    data["% particule"] = data.particules/tquantity
    data["% echantillon"] = data.echantillon/nsamples
    
    return data

## Données mises en forme 

In [3]:
work_data.head().style.set_table_styles(table_css_styles)

,Plage,echantillon,orientation,position,substrat,frequentation,situation,distance,objet,particules
0,Amphion,74_Amp_1,NE,1,4,3,1,1,fibres,97
1,Amphion,74_Amp_10,NNE,2,4,3,1,1,fibres,140
2,Amphion,74_Amp_2,NNE,1,4,3,1,1,fibres,121
3,Amphion,74_Amp_3,NE,1,4,3,1,1,fibres,31
4,Amphion,74_Amp_4,NNE,1,4,3,1,1,fibres,179


In [4]:
agg_ = {"particules":"sum", "echantillon":"nunique"}

position_total = work_data.groupby("position", as_index=False).agg(agg_)
substrat_total = work_data.groupby("substrat", as_index=False).agg(agg_)
frequentation_total = work_data.groupby("frequentation", as_index=False).agg(agg_)
situation_total = work_data.groupby("situation", as_index=False).agg(agg_)
orient_total = work_data.groupby("orientation", as_index=False).agg(agg_)
particle_type_total = work_data.groupby("objet", as_index=False).agg(agg_)
distance_total =  work_data.groupby("distance", as_index=False).agg(agg_)

## Variables indépendantes

```{figure} resources/maps/intro_map.jpeg
---
name: basicmap-fig
---
Les sites d'échantillonnage de 2022
```

### Lieu d'echantillonage

In [5]:
beaches = pd.read_csv("data/end_pipe/asl_beaches.csv")
beaches.set_index("Plage", inplace=True, drop=True)
beaches.index.name = None
beaches.style.format(precision=2).set_table_styles(table_css_styles)

,frequentation,situation,distance,orientation,x,y
Amphion,3,1,1,NE,46.40,6.53
Anthy,2,1,1,NNO,46.35,6.40
Excenevex,3,1,1,NE,46.35,6.36
Lugrin,2,1,2,NNE,46.40,6.67
Meillerie,1,2,3,N,46.41,6.72
Saint-Disdille,2,1,1,N,46.40,6.50
Tougues,3,1,2,NO,46.32,6.26
Baby Plage,3,2,2,NNE,46.21,6.16
Hermance,3,2,2,OSO,46.30,6.24
Port Choiseul,3,2,2,NE,46.29,6.17


### Position

In [6]:
pos_total = add_weight_of_samples(position_total)
pos_total = add_labels_display(pos_total, "position", name_the_zones)
pos_total

,particules,echantillon,% particule,% echantillon
ligne-d'eau,12969,111,0.33,0.51
plage-seche,26120,106,0.67,0.49


### Substrat

In [7]:
sub_total = add_weight_of_samples(substrat_total)

add_labels_display(sub_total, "substrat", name_the_substrate)

,particules,echantillon,% particule,% echantillon
sable fin,19345,114,0.49,0.53
sable grosssier,4253,28,0.11,0.13
gravier,9732,29,0.25,0.13
galet,5759,46,0.15,0.21


### Fréquentation

In [8]:
freq_total = add_weight_of_samples(frequentation_total)
add_labels_display(freq_total, "frequentation", frequentation_name)

,particules,echantillon,% particule,% echantillon
faible,587,8,0.02,0.04
moyenne,9910,37,0.25,0.17
élévée,28592,172,0.73,0.79


### Situation

In [9]:
sit_total =  add_weight_of_samples(situation_total)
add_labels_display(sit_total, "situation", situation_name)

,particules,echantillon,% particule,% echantillon
campagne,26098,140,0.67,0.65
urbain,12991,77,0.33,0.35


### Distance parking

In [10]:
dis_total = add_weight_of_samples(distance_total)
add_labels_display(dis_total, "distance", name_the_distance)

,particules,echantillon,% particule,% echantillon
< 100m,16443,93,0.42,0.43
100 - 500m,21430,110,0.55,0.51
500 - 1000m,76,2,0.00,0.01
> 1000m,1140,12,0.03,0.06


### Orientation de la plage

In [11]:
ori_total =  add_weight_of_samples(orient_total)
ori_total.set_index("orientation", drop=True, inplace=True)
ori_total.index.name = None
ori_total.style.format(precision=2).set_table_styles(table_css_styles)

,particules,echantillon,% particule,% echantillon
E,40,1,0.00,0.00
ENE,271,2,0.01,0.01
ESE,399,4,0.01,0.02
N,2380,14,0.06,0.06
NE,8158,42,0.21,0.19
NNE,1104,9,0.03,0.04
NNO,1339,10,0.03,0.05
NO,5675,25,0.15,0.12
O,1487,7,0.04,0.03
ONO,724,6,0.02,0.03


## Variables dépendantes

### Fragments plastiques par forme

In [12]:
par_total =  add_weight_of_samples(particle_type_total)
add_labels_display(par_total, "objet", particle_groups)

,particules,echantillon,% particule,% echantillon
particule rigide,5105,217,0.13,1.00
fibre,29909,217,0.77,1.00
particule souple,4075,217,0.10,1.00


### Fragments plastiques par forme et couleur

In [14]:
p_t_c = c[name_the_particles.keys()].sum(axis=0)
p_t_cdf = pd.DataFrame({"couleur et type":p_t_c.index, "compte":p_t_c.values})
p_t_cdf["% particule"] = p_t_cdf.compte/tquantity
p_t_cdf["couleur et type"] = p_t_cdf["couleur et type"].apply(lambda x: name_the_particles[x])
p_t_cdf.set_index("couleur et type", drop=True, inplace=True)
p_t_cdf.index.name = None
p_t_cdf["compte"] = p_t_cdf.compte.astype("int")
p_t_cdf.rename(columns={"compte":"particules"}, inplace=True)
p_t_cdf.style.format(precision=2).set_table_styles(table_css_styles)

,particules,% particule
fibre rouge,2364,0.06
fibre bleu,4060,0.10
fibre jaune,400,0.01
fibre transparent,16920,0.43
fibre noire,5892,0.15
fibre autre,273,0.01
particule rigide rouge,955,0.02
particule rigide bleu,1400,0.04
particule rigide jaune,508,0.01
particule rigide transparent,1202,0.03
